# Assignment 1: Dimensionality Reduction

This notebook is a starter template for your assignment. Please fill in the required functions and complete the analysis as described in the assignment instructions.


In [1]:
# libraries
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load the data
# You can use numpy to load the data from a file.

def load_data(file_path):
    """
    Load the dataset from a given file path.
    Args:
        file_path: str, path to the data file
    Returns:
        data: numpy array of shape (n_samples, n_features)
    """
    data = np.loadtxt(file_path)
    return data

# Example usage:
data = load_data('./data/Asgmnt1_data.txt')
print('Data shape:', data.shape)

Data shape: (16000, 128)


In [3]:
## Normalization
def normalize_data(X):
    """
    Normalize the data by row
    :param X: numpy array
    :return: row-normalized numpy array
    """
    mean = np.mean(X, axis=1, keepdims=True)
    std = np.std(X, axis=1, keepdims=True)
    return (X - mean) / (std + 1e-8)  # Avoid division by zero

def euclidean_distance_matrix(X):
    """
    Compute the pairwise Euclidean distance matrix for X.
    Args:
        X: numpy array of shape (n_samples, n_features)
    Returns:
        dist_matrix: numpy array of shape (n_samples, n_samples)
    """
    X = np.asarray(X)
    sum_X = np.sum(np.square(X), axis=1)
    dist_matrix = np.sqrt(np.maximum(sum_X[:, None] + sum_X[None, :] - 2 * np.dot(X, X.T), 0))
    return dist_matrix

# Example usage (you can uncomment this after implementing for testing):
# dist_matrix = euclidean_distance(data_norm[:100])
# print('Distance matrix shape:', dist_matrix.shape)

def haar_matrix(n):
    if n == 1:
        return np.array([[1]])
    H = haar_matrix(n // 2)
    top = np.kron(H, np.array([[1.0, 1.0]])) 
    bottom = np.kron(np.eye(len(H)), [1, -1])
    return np.vstack((top, bottom)) #/ np.sqrt(2.0)


def wavelet_transform(X, haar_n, reduced_n):
    """
    Apply Haar wavelet transform to X using an n x n Haar matrix.
    Args:
        X: numpy array of shape (n_samples, n_features)
        haar_n: int, number of features (should match X.shape[1])
    Returns:
        X_wavelet: numpy array of shape (n_samples, n_features)
    """
    H = haar_matrix(haar_n)
    norm_factor = np.count_nonzero(H, axis=1)  # Normalization factor
    X_wavelet = np.dot(X, H.T) / norm_factor
    return X_wavelet[:, :reduced_n]

# Example usage (you can uncomment this after implementing for testing):
# H = haar_matrix(data_norm.shape[1])
# data_wavelet = wavelet_transform(data_norm, data_norm.shape[1])
# print('Wavelet-transformed data shape:', data_wavelet.shape)

def pca(X, n_components):
    """
    Perform PCA on X and return the projected data.
    Args:
        X: numpy array of shape (n_samples, n_features)
        n_components: int, number of principal components to keep
    Returns:
        X_pca: numpy array of shape (n_samples, n_components)
    """
    # z-normalize the data
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    X_centered = (X - mean) / std  # Avoid division by zero
    X_centered = np.nan_to_num(X_centered)
    # Center the data
    # X_centered = X - np.mean(X, axis=0)
    # cov = np.cov(X_centered, rowvar=False)
    cov = np.dot(X_centered.T, X_centered)
    eigvals, eigvecs = np.linalg.eigh(cov)
    idx = np.argsort(eigvals)[::-1]
    eigvecs = eigvecs[:, idx]
    eigvecs = eigvecs[:, :n_components]
    X_pca = np.dot(X_centered, eigvecs)
    return X_pca

# Example usage (uncomment after implementing):
# data_pca = pca(data_znorm, 4)
# print('PCA-transformed data shape:', data_pca.shape)


In [4]:
# Your implementation for measuring elapsed time goes here
import time
data_norm = normalize_data(data)
print('Normalized data shape:', data_norm.shape)
# Use a subset for demonstration if the dataset is large (e.g., first 100 samples)
subset = 16000
X_orig = data_norm[:subset]
X_wavelet = wavelet_transform(X_orig, haar_n=X_orig.shape[1], reduced_n=4)
X_pca = pca(X_orig, n_components=4)

# 1. Time for original dataset
start = time.time()
dist_orig = euclidean_distance_matrix(X_orig)
time_orig = time.time() - start
print(f"Time for Euclidean distance (original): {time_orig:.4f} seconds")

# 2. Time for wavelet-transformed dataset
start = time.time()
dist_wavelet = euclidean_distance_matrix(X_wavelet)
time_wavelet = time.time() - start
print(f"Time for Euclidean distance (wavelet): {time_wavelet:.4f} seconds")

# 3. Time for PCA-transformed dataset
start = time.time()
dist_pca = euclidean_distance_matrix(X_pca)
time_pca = time.time() - start
print(f"Time for Euclidean distance (PCA): {time_pca:.4f} seconds")

Normalized data shape: (16000, 128)
Time for Euclidean distance (original): 1.6705 seconds
Time for Euclidean distance (wavelet): 1.2662 seconds
Time for Euclidean distance (PCA): 1.6326 seconds


In [5]:
# Your implementation for distance relation consistency goes here
import random

def relationship_consistency(dist_orig, dist_reduced, num_samples=100000, seed=None):
    """
    Compare pairwise distance relationships between original and reduced distance matrices.
    Returns the fraction of relationships preserved.
    Args:
        dist_orig: numpy array, shape (n, n), original distance matrix
        dist_reduced: numpy array, shape (n, n), reduced distance matrix
        num_samples: int, number of random triplets to sample
        seed: int, random seed for reproducibility
    Returns:
        consistency: float, fraction of preserved relationships
    """
    if seed is not None:
        np.random.seed(seed)
    n = dist_orig.shape[0]
    count = 0
    preserved = 0
    for _ in range(num_samples):
        # Randomly pick anchor, B, C (all different)
        a, b, c = np.random.choice(n, 3, replace=False)
        rel_orig = dist_orig[a, b] > dist_orig[a, c]
        rel_reduced = dist_reduced[a, b] > dist_reduced[a, c]
        if rel_orig == rel_reduced:
            preserved += 1
        count += 1
    return preserved / count

# Example usage:
cons_wavelet = relationship_consistency(dist_orig, dist_wavelet)
cons_pca = relationship_consistency(dist_orig, dist_pca)
print(f"Wavelet consistency: {cons_wavelet:.4f}")
print(f"PCA consistency: {cons_pca:.4f}")

Wavelet consistency: 0.6012
PCA consistency: 0.6224


In [6]:
import numpy as np

eval_data = np.array([
    [1, 1, 1, 1, 1, 1, 1, 1],
    [1.1, 0.9, 1, 1, 1, 1, 1, 1],
    [0.95, 1.05, 1, 1, 1, 1, 1, 1],
    [1, 1, 1.1, 0.9, 1, 1, 1, 1],
    [-1, -1, -1, -1, -1, -1, -1, -1],
    [-1.1, -0.9, -1, -1, -1, -1, -1, -1],
    [-0.95, -1.05, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1.1, -0.9, -1, -1, -1, -1]
])

def all_close(actual, expected, tol=1e-4, no_sign=False):

    if isinstance(expected, list):
        correct_1 = any([np.allclose(actual, e, atol=tol) for e in expected])
        correct_2 = any([np.allclose(np.abs(actual), np.abs(e), atol=tol) for e in expected]) if no_sign else False
        correct = correct_1 or correct_2
    else:
        correct = np.allclose(actual, expected, atol=tol) if not no_sign else np.allclose(np.abs(actual), np.abs(expected), atol=tol)
    
    if not correct:
        print("The function is NOT working correctly.")
        print("Expected result:")
        print(expected)
        print("Actual result:")
        print(actual)
    else:
        print("The function is working correctly.")
        print("Actual result:")
        print(actual)

In [7]:
# DATA NORMALIZATION
expected_result = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0],
    [2, -2, 0, 0, 0, 0, 0, 0],
    [-2, 2, 0, 0, 0, 0, 0, 0],
    [0, 0, 2, -2, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [-2, 2, 0, 0, 0, 0, 0, 0],
    [2, -2, 0, 0, 0, 0, 0, 0],
    [0, 0, -2, 2, 0, 0, 0, 0]
])


actual_result = normalize_data(eval_data)
# compare the normalized data with the expected result
all_close(actual_result, expected_result)

The function is working correctly.
Actual result:
[[ 0.         0.         0.         0.         0.         0.
   0.         0.       ]
 [ 1.9999996 -1.9999996  0.         0.         0.         0.
   0.         0.       ]
 [-1.9999992  1.9999992  0.         0.         0.         0.
   0.         0.       ]
 [ 0.         0.         1.9999996 -1.9999996  0.         0.
   0.         0.       ]
 [ 0.         0.         0.         0.         0.         0.
   0.         0.       ]
 [-1.9999996  1.9999996  0.         0.         0.         0.
   0.         0.       ]
 [ 1.9999992 -1.9999992  0.         0.         0.         0.
   0.         0.       ]
 [ 0.         0.        -1.9999996  1.9999996  0.         0.
   0.         0.       ]]


In [8]:
# DISTANCE MATRIX
expected_result = np.array(
    [
        [0.    , 0.1414, 0.0707, 0.1414, 5.6569, 5.6586, 5.6573, 5.6586],
        [0.1414, 0.    , 0.2121, 0.2   , 5.6586, 5.6639, 5.6573, 5.6604],
        [0.0707, 0.2121, 0.    , 0.1581, 5.6573, 5.6573, 5.6586, 5.6591],
        [0.1414, 0.2   , 0.1581, 0.    , 5.6586, 5.6604, 5.6591, 5.6639],
        [5.6569, 5.6586, 5.6573, 5.6586, 0.    , 0.1414, 0.0707, 0.1414],
        [5.6586, 5.6639, 5.6573, 5.6604, 0.1414, 0.    , 0.2121, 0.2   ],
        [5.6573, 5.6573, 5.6586, 5.6591, 0.0707, 0.2121, 0.    , 0.1581],
        [5.6586, 5.6604, 5.6591, 5.6639, 0.1414, 0.2   , 0.1581, 0.    ]
    ]
)

actual_result = euclidean_distance_matrix(eval_data)
# compare the distance matrix with the expected result
all_close(actual_result, expected_result)

The function is working correctly.
Actual result:
[[0.         0.14142136 0.07071068 0.14142136 5.65685425 5.65862174
  5.65729617 5.65862174]
 [0.14142136 0.         0.21213203 0.2        5.65862174 5.6639209
  5.65729617 5.66038868]
 [0.07071068 0.21213203 0.         0.15811388 5.65729617 5.65729617
  5.65862174 5.65906353]
 [0.14142136 0.2        0.15811388 0.         5.65862174 5.66038868
  5.65906353 5.6639209 ]
 [5.65685425 5.65862174 5.65729617 5.65862174 0.         0.14142136
  0.07071068 0.14142136]
 [5.65862174 5.6639209  5.65729617 5.66038868 0.14142136 0.
  0.21213203 0.2       ]
 [5.65729617 5.65729617 5.65862174 5.65906353 0.07071068 0.21213203
  0.         0.15811388]
 [5.65862174 5.66038868 5.65906353 5.6639209  0.14142136 0.2
  0.15811388 0.        ]]


In [9]:
#  HAAR MATRIX
expected_result = np.array([
    [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1., -1., -1., -1., -1.],
    [ 1.,  1., -1., -1.,  0.,  0., -0., -0.],
    [ 0.,  0., -0., -0.,  1.,  1., -1., -1.],
    [ 1., -1.,  0., -0.,  0., -0.,  0., -0.],
    [ 0., -0.,  1., -1.,  0., -0.,  0., -0.],
    [ 0., -0.,  0., -0.,  1., -1.,  0., -0.],
    [ 0., -0.,  0., -0.,  0., -0.,  1., -1.]])

actual_result = haar_matrix(8)
# compare the Haar matrix with the expected result
all_close(actual_result, expected_result)

The function is working correctly.
Actual result:
[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1. -1. -1. -1. -1.]
 [ 1.  1. -1. -1.  0.  0. -0. -0.]
 [ 0.  0. -0. -0.  1.  1. -1. -1.]
 [ 1. -1.  0. -0.  0. -0.  0. -0.]
 [ 0. -0.  1. -1.  0. -0.  0. -0.]
 [ 0. -0.  0. -0.  1. -1.  0. -0.]
 [ 0. -0.  0. -0.  0. -0.  1. -1.]]


In [10]:
# wavelet
expected_result = np.array([
    [ 0.,  0.,  0.,  0.],
    [ 0.,  0.,  0.,  0.],
    [ 0.,  0.,  0.,  0.],
    [ 0.,  0., -0.,  0.],
    [ 0.,  0.,  0.,  0.],
    [-0., -0., -0.,  0.],
    [ 0.,  0.,  0.,  0.],
    [-0., -0.,  0.,  0.]]
    )

data = normalize_data(eval_data)
actual_result = wavelet_transform(data, haar_n=data.shape[1], reduced_n=4)
# compare the wavelet transformed data with the expected result
all_close(actual_result, expected_result)

The function is working correctly.
Actual result:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.77555756e-16  2.77555756e-16  5.55111512e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.77555756e-16  2.77555756e-16 -5.55111512e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.77555756e-16 -2.77555756e-16 -5.55111512e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.77555756e-16 -2.77555756e-16  5.55111512e-16  0.00000000e+00]]


In [11]:
# PCA
expected_result_center = np.array([
    [ 0.    ,  0.    ,  0.    ,  0.    ],
    [ 2.8284,  0.    ,  0.    ,  0.    ],
    [-2.8284,  0.    ,  0.    ,  0.    ],
    [ 0.    ,  2.8284,  0.    ,  0.    ],
    [ 0.    ,  0.    ,  0.    ,  0.    ],
    [-2.8284,  0.    , -0.    ,  0.    ],
    [ 2.8284,  0.    , -0.    ,  0.    ],
    [ 0.    , -2.8284,  0.    , -0.    ]])

expected_result_2_znorm = np.array([
    [ 0.    ,  0.    ,  0.    ,  0.    ],
    [ 2.    ,  0.    ,  0.    ,  0.    ],
    [-2.    ,  0.    ,  0.    ,  0.    ],
    [ 0.    ,  2.8284,  0.    ,  0.    ],
    [ 0.    ,  0.    ,  0.    ,  0.    ],
    [-2.    ,  0.    , -0.    ,  0.    ],
    [ 2.    ,  0.    , -0.    ,  0.    ],
    [ 0.    , -2.8284,  0.    , -0.    ]])

data = normalize_data(eval_data)
actual_result = pca(data, n_components=4)
# compare the PCA transformed data with the expected result
all_close(actual_result, [expected_result_center, expected_result_2_znorm], no_sign=True)

The function is working correctly.
Actual result:
[[ 0.          0.          0.          0.        ]
 [-2.0000002   0.          0.          0.        ]
 [ 1.9999998   0.          0.          0.        ]
 [ 0.         -2.82842712  0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 2.0000002   0.          0.          0.        ]
 [-1.9999998   0.          0.          0.        ]
 [ 0.          2.82842712  0.          0.        ]]


/tmp/ipykernel_71014/220995127.py:69: RuntimeWarning: invalid value encountered in divide
  X_centered = (X - mean) / std  # Avoid division by zero


In [12]:
# use sklearn pca for comparison
from sklearn.decomposition import PCA as sklearnPCA
data = normalize_data(eval_data)
sklearn_pca = sklearnPCA(n_components=4)
sklearn_pca.fit(data)
actual_result = sklearn_pca.transform(data)
# compare the sklearn PCA transformed data with the expected result
all_close(actual_result, [expected_result_center, expected_result_2_znorm], no_sign=True)

The function is working correctly.
Actual result:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.82842656e+00  0.00000000e+00  7.81347640e-16  0.00000000e+00]
 [-2.82842599e+00  0.00000000e+00  8.56493357e-16  0.00000000e+00]
 [ 0.00000000e+00  2.82842656e+00  0.00000000e+00  1.15213870e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.82842656e+00  0.00000000e+00 -7.81347640e-16  0.00000000e+00]
 [ 2.82842599e+00  0.00000000e+00 -8.56493357e-16  0.00000000e+00]
 [ 0.00000000e+00 -2.82842656e+00  0.00000000e+00 -1.15213870e-16]]


In [13]:
# Verify wavelet with pywavelet
import pywt

data = normalize_data(eval_data)
coeffs = pywt.wavedec(data, 'haar', level=1)
# Reconstruct the wavelet coefficients to get the transformed data
actual_result = pywt.waverec(coeffs, 'haar')[:, :4]
# compare the pywavelet transformed data with the expected result
all_close(actual_result, expected_result)

The function is NOT working correctly.
Expected result:
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -0.  0.]
 [ 0.  0.  0.  0.]
 [-0. -0. -0.  0.]
 [ 0.  0.  0.  0.]
 [-0. -0.  0.  0.]]
Actual result:
[[ 0.         0.         0.         0.       ]
 [ 1.9999996 -1.9999996  0.         0.       ]
 [-1.9999992  1.9999992  0.         0.       ]
 [ 0.         0.         1.9999996 -1.9999996]
 [ 0.         0.         0.         0.       ]
 [-1.9999996  1.9999996  0.         0.       ]
 [ 1.9999992 -1.9999992  0.         0.       ]
 [ 0.         0.        -1.9999996  1.9999996]]


In [14]:
normalize_data(eval_data)

array([[ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ],
       [ 1.9999996, -1.9999996,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ],
       [-1.9999992,  1.9999992,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  1.9999996, -1.9999996,  0.       ,
         0.       ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ],
       [-1.9999996,  1.9999996,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ],
       [ 1.9999992, -1.9999992,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ],
       [ 0.       ,  0.       , -1.9999996,  1.9999996,  0.       ,
         0.       ,  0.       ,  0.       ]])